In [ ]:

# 如果你未安装以下库，请先运行这些命令（只需运行一次）：
# !pip install dash pandas plotly jupyter-dash

from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px

# 读取数据
data = pd.read_csv(
    'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv'
)

# 初始化 JupyterDash 应用
app = JupyterDash(__name__)

year_list = [i for i in range(1980, 2024)]

app.layout = html.Div([
    html.H1("Automobile Sales Statistics Dashboard", style={'textAlign': 'center', 'color': '#503D36', 'fontSize': 24}),

    html.Label("Select Statistics:"),
    dcc.Dropdown(
        id='dropdown-statistics',
        options=[
            {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
            {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
        ],
        placeholder='Select Statistics',
        style={'width': '80%', 'padding': '3px', 'fontSize': '20px', 'textAlignLast': 'center'}
    ),

    dcc.Dropdown(
        id='select-year',
        options=[{'label': i, 'value': i} for i in year_list],
        placeholder='Select a year',
        style={'width': '80%', 'padding': '3px', 'fontSize': '20px', 'textAlignLast': 'center'}
    ),

    html.Div(id='output-container', className='chart-grid', style={'display': 'flex', 'flexDirection': 'column'})
])

@app.callback(
    Output('select-year', 'disabled'),
    Input('dropdown-statistics', 'value')
)
def update_year_dropdown(stat_type):
    return stat_type != 'Yearly Statistics'

@app.callback(
    Output('output-container', 'children'),
    [Input('select-year', 'value'),
     Input('dropdown-statistics', 'value')]
)
def update_output(input_year, stat_type):
    if stat_type == 'Recession Period Statistics':
        recession_data = data[data['Recession'] == 1]

        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(figure=px.line(yearly_rec, x='Year', y='Automobile_Sales',
                                            title="Average Automobile Sales over Recession Period"))

        avg_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 = dcc.Graph(figure=px.line(avg_sales, x='Vehicle_Type', y='Automobile_Sales',
                                            title='Average Vehicles Sold by Type during Recession'))

        exp_rec = recession_data.groupby('Vehicle_Type')['Total_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(figure=px.pie(exp_rec, values='Total_Expenditure', names='Vehicle_Type',
                                           title='Expenditure Share by Vehicle Type during Recessions'))

        unem_sales = recession_data.groupby('Vehicle_Type').agg({
            'Unemployment_Rate': 'mean',
            'Automobile_Sales': 'mean'
        }).reset_index()
        R_chart4 = dcc.Graph(figure=px.bar(unem_sales, x='Unemployment_Rate', y='Automobile_Sales',
                                           color='Vehicle_Type',
                                           title='Unemployment Rate vs Sales by Vehicle Type'))

        return [
            html.Div([R_chart1, R_chart2], style={'display': 'flex'}),
            html.Div([R_chart3, R_chart4], style={'display': 'flex'})
        ]

    elif input_year and stat_type == 'Yearly Statistics':
        yearly_data = data[data['Year'] == input_year]

        yas = data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(figure=px.line(yas, x='Year', y='Automobile_Sales',
                                            title='Yearly Average Automobile Sales'))

        monthly_sales = yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        Y_chart2 = dcc.Graph(figure=px.line(monthly_sales, x='Month', y='Automobile_Sales',
                                            title='Monthly Automobile Sales in {}'.format(input_year)))

        avg_veh = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph(figure=px.bar(avg_veh, x='Vehicle_Type', y='Automobile_Sales',
                                           title='Avg Vehicles Sold by Type in {}'.format(input_year)))

        adv_exp = yearly_data.groupby('Vehicle_Type')['Advertisement_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(figure=px.pie(adv_exp, values='Advertisement_Expenditure', names='Vehicle_Type',
                                           title='Ad Expenditure by Vehicle Type in {}'.format(input_year)))

        return [
            html.Div([Y_chart1, Y_chart2], style={'display': 'flex'}),
            html.Div([Y_chart3, Y_chart4], style={'display': 'flex'})
        ]
    else:
        return []

# 在 notebook 中运行服务器
app.run_server(mode='inline', debug=True)
